In [1]:
# [+] SparkSession 설정
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

In [2]:
# 샘플 데이터: 한신대 맛집 및 대표메뉴
people = [
    ('이선민', 23),
    ('장원영', 20),
    ('이영지', 22),
    ('전지현', 43),
    ('차은우', 28),
    ('정우성', 45),
    ('공유', 66),
    ('김성철', 31)
]


In [3]:
# [+] 스키마 정의
schema = ['name', 'age']

In [4]:
# [+] 데이터프레임 생성
df = spark.createDataFrame(people, schema)

In [5]:
# [+] 데이터프레임 출력
df.show()

+------+---+
|  name|age|
+------+---+
|이선민| 23|
|장원영| 20|
|이영지| 22|
|전지현| 43|
|차은우| 28|
|정우성| 45|
|  공유| 66|
|김성철| 31|
+------+---+



In [6]:
# [+] 데이터프레임 스키마 출력
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [22]:
# [+] Temporary View 생성
df.createOrReplaceTempView('people_table')

### User Defined Function 생성
1. translate(): Google Translation API를 이용하여 한글 식당 이름을 영문으로 번역하는 함수
2. scale_score(): 5점 만점 평점을 100점 스케일로 변환하는 함수

In [8]:
# Google translation 라이브러리 설치
!pip install googletrans==4.0.0-rc1

In [23]:
# Google translator 임포트
from googletrans import Translator

#객체 생성
peopleAge = Translator()

In [25]:
def age_category(age):
    if age < 35:
        return 'young'
    elif age >=35 and age <60:
        return 'adult'
    elif age >= 60:
        return 'senior'

In [26]:
# UDF 등록
spark.udf.register('age_category', age_category)

<function __main__.age_category(age)>

In [45]:
spark.sql(
        "SELECT name, \
        age, \
        age_category(age) AS age_category_show\
        FROM people_table").show()

+------+---+-----------------+
|  name|age|age_category_show|
+------+---+-----------------+
|이선민| 23|            young|
|장원영| 20|            young|
|이영지| 22|            young|
|전지현| 43|            adult|
|차은우| 28|            young|
|정우성| 45|            adult|
|  공유| 66|           senior|
|김성철| 31|            young|
+------+---+-----------------+

